In [0]:
%run "/Workspace/Users/sundaramloganathan@outlook.com/includes/storage_acc_config"

In [0]:
%run "../includes/folder_path"

In [0]:
dbutils.widgets.text("p_owner_name","")
v_owner_name = dbutils.widgets.get("p_owner_name")

In [0]:
from pyspark.sql.types import StructField, StringType, IntegerType, DoubleType, StructType, TimestampType


qual_schema = StructType(fields=[
    StructField("qualifyId", IntegerType(), False),
    StructField("raceId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("q1", StringType(), True),
    StructField("q2", StringType(), True),
    StructField("q3", StringType(), True)])


In [0]:
qual_df = spark.read.options(multiLine=True).schema(qual_schema).json(f"{bronze_folder_path}/qualifying")

In [0]:
from pyspark.sql.functions import current_timestamp, lit


qual_upd_df = qual_df.withColumnRenamed("raceId", "race_id") \
                     .withColumnRenamed("driverId", "driver_id") \
                     .withColumnRenamed("constructorId", "constructor_id") \
                     .withColumnRenamed("qualifyId", "qualify_id") \
                     .withColumn("ingested_by",lit(v_owner_name)) \
                     .withColumn("ingestion_date", current_timestamp())

In [0]:
qual_upd_df.write.mode("overwrite").parquet(f"{silver_folder_path}/qualifying/")